### Save and restore models

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/save_and_restore_models"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_restore_models.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_restore_models.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Model progress can be saved during—and after—training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work. When publishing research models and techniques, most machine learning practitioners share:

* code to create the model, and
* the trained weights, or parameters, for the model

Sharing this data helps others understand how the model works and try it themselves with new data.

Caution: Be careful with untrusted code—TensorFlow models are code. See [Using TensorFlow Securely](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md) for details.

#### Options

There are different ways to save TensorFlow models—depending on the API you're using. This guide uses [tf.keras](https://www.tensorflow.org/guide/keras), a high-level API to build and train models in TensorFlow. For other approaches, see the TensorFlow  [Save and Restore](https://www.tensorflow.org/guide/saved_model) guide or [Saving in eager](https://www.tensorflow.org/guide/eager#object-based_saving).


### Setup

#### Installs and imports

Install and import TensorFlow and dependencies:

In [1]:
!pip install h5py pyyaml

    100% |████████████████████████████████| 276kB 4.4MB/s ta 0:00:01
  Stored in directory: /Users/shawlu/Library/Caches/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml


#### Get an example dataset

We'll use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to train our model to demonstrate saving weights. To speed up these demonstration runs, only use the first 1000 examples:

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.14.1-dev20190429'

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

#### Define a model

Let's build a simple model we'll use to demonstrate saving and loading weights.

In [4]:
# Returns a short sequential model
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation=tf.keras.activations.softmax)
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])

    return model


# Create a basic model instance
model = create_model()
model.summary()

W0429 23:42:46.484585 4562052544 deprecation.py:506] From /Users/shawlu/anaconda3/envs/deep/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### Save checkpoints during training

The primary use case is to automatically save checkpoints *during* and at *the end* of training. This way you can use a trained model without having to retrain it, or pick-up training where you left of—in case the training process was interrupted.

`tf.keras.callbacks.ModelCheckpoint` is a callback that performs this task. The callback takes a couple of arguments to configure checkpointing.

#### Checkpoint callback usage

Train the model and pass it the `ModelCheckpoint` callback:

In [5]:
checkpoint_path = "outputs/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 768/1000 [======================>.......] - ETA: 0s - loss: 1.3308 - acc: 0.6380
Epoch 00001: saving model to outputs/cp.ckpt
1000/1000 [==============================] - 0s 374us/sample - loss: 1.1911 - acc: 0.6650 - val_loss: 0.7438 - val_acc: 0.7710
Epoch 2/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.4272 - acc: 0.8841
Epoch 00002: saving model to outputs/cp.ckpt
1000/1000 [==============================] - 0s 147us/sample - loss: 0.4249 - acc: 0.8780 - val_loss: 0.5304 - val_acc: 0.8410
Epoch 3/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.2846 - acc: 0.9300
Epoch 00003: saving model to outputs/cp.ckpt
1000/1000 [==============================] - 0s 139us/sample - loss: 0.2859 - acc: 0.9320 - val_loss: 0.4637 - val_acc: 0.8570
Epoch 4/10
 640/1000 [==================>...........] - ETA: 0s - loss: 0.1886 - acc: 0.9594
Epoch 00004: saving model to outputs/cp.ckpt
1000/1000 [========

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:

In [6]:
!ls {checkpoint_dir}

102_mnist_fashion.h5        cp.ckpt.data-00001-of-00002
checkpoint                  cp.ckpt.index
cp.ckpt.data-00000-of-00002


Create a new, untrained model. When restoring a model from only weights, you must have a model with the same architecture as the original model. Since it's the same model architecture, we can share weights despite that it's a different *instance* of the model.

Now rebuild a fresh, untrained model, and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [7]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 65us/sample - loss: 2.3155 - acc: 0.1440
Untrained model, accuracy: 14.40%


Then load the weights from the checkpoint, and re-evaluate:

In [8]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 31us/sample - loss: 0.4300 - acc: 0.8660
Restored model, accuracy: 86.60%


#### Checkpoint callback options

The callback provides several options to give the resulting checkpoints unique names, and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every 5-epochs:


In [9]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "outputs/training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)

W0429 23:43:58.362816 4562052544 callbacks.py:856] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.



Epoch 00005: saving model to outputs/training_2/cp-0005.ckpt

Epoch 00010: saving model to outputs/training_2/cp-0010.ckpt

Epoch 00015: saving model to outputs/training_2/cp-0015.ckpt

Epoch 00020: saving model to outputs/training_2/cp-0020.ckpt

Epoch 00025: saving model to outputs/training_2/cp-0025.ckpt

Epoch 00030: saving model to outputs/training_2/cp-0030.ckpt

Epoch 00035: saving model to outputs/training_2/cp-0035.ckpt

Epoch 00040: saving model to outputs/training_2/cp-0040.ckpt

Epoch 00045: saving model to outputs/training_2/cp-0045.ckpt

Epoch 00050: saving model to outputs/training_2/cp-0050.ckpt


Now, look at the resulting checkpoints and choose the latest one:

In [10]:
! ls {checkpoint_dir}

checkpoint                       cp-0025.ckpt.data-00001-of-00002
cp-0000.ckpt.data-00000-of-00002 cp-0025.ckpt.index
cp-0000.ckpt.data-00001-of-00002 cp-0030.ckpt.data-00000-of-00002
cp-0000.ckpt.index               cp-0030.ckpt.data-00001-of-00002
cp-0005.ckpt.data-00000-of-00002 cp-0030.ckpt.index
cp-0005.ckpt.data-00001-of-00002 cp-0035.ckpt.data-00000-of-00002
cp-0005.ckpt.index               cp-0035.ckpt.data-00001-of-00002
cp-0010.ckpt.data-00000-of-00002 cp-0035.ckpt.index
cp-0010.ckpt.data-00001-of-00002 cp-0040.ckpt.data-00000-of-00002
cp-0010.ckpt.index               cp-0040.ckpt.data-00001-of-00002
cp-0015.ckpt.data-00000-of-00002 cp-0040.ckpt.index
cp-0015.ckpt.data-00001-of-00002 cp-0045.ckpt.data-00000-of-00002
cp-0015.ckpt.index               cp-0045.ckpt.data-00001-of-00002
cp-0020.ckpt.data-00000-of-00002 cp-0045.ckpt.index
cp-0020.ckpt.data-00001-of-00002 cp-0050.ckpt.data-00000-of-00002
cp-0020.ckpt.index               cp-0050.ckpt.data-00001-of-00002
cp-0025.ckpt.d

In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'outputs/training_2/cp-0050.ckpt'

Note: the default tensorflow format only saves the 5 most recent checkpoints.

To test, reset the model and load the latest checkpoint:

In [12]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 70us/sample - loss: 0.4708 - acc: 0.8760
Restored model, accuracy: 87.60%


### What are these files?

The above code stores the weights to a collection of [checkpoint](https://www.tensorflow.org/guide/saved_model#save_and_restore_variables)-formatted files that contain only the trained weights in a binary format. Checkpoints contain:
* One or more shards that contain your model's weights.
* An index file that indicates which weights are stored in a which shard.

If you are only training a model on a single machine, you'll have one shard with the suffix: `.data-00000-of-00001`

### Manually save weights

Above you saw how to load the weights into a model.

Manually saving the weights is just as simple, use the `Model.save_weights` method.

In [13]:
# Save the weights
model.save_weights('./outputs/checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./outputs/checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 70us/sample - loss: 0.4708 - acc: 0.8760
Restored model, accuracy: 87.60%


### Save the entire model

The entire model can be saved to a file that contains the weight values, the model's configuration, and even the optimizer's configuration (depends on set up). This allows you to checkpoint a model and resume training later—from the exact same state—without access to the original code.

Saving a fully-functional model is very useful—you can load them in TensorFlow.js ([HDF5](https://js.tensorflow.org/tutorials/import-keras.html), [Saved Model](https://js.tensorflow.org/tutorials/import-saved-model.html)) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite ([HDF5](https://www.tensorflow.org/lite/convert/python_api#exporting_a_tfkeras_file_), [Saved Model](https://www.tensorflow.org/lite/convert/python_api#exporting_a_savedmodel_))

### As an HDF5 file

Keras provides a basic save format using the [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standard. For our purposes, the saved model can be treated as a single binary blob.

In [14]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('outputs/my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 149us/sample - loss: 1.1518 - acc: 0.6840
Epoch 2/5
1000/1000 [==============================] - 0s 60us/sample - loss: 0.4246 - acc: 0.8760
Epoch 3/5
1000/1000 [==============================] - 0s 62us/sample - loss: 0.2802 - acc: 0.9240
Epoch 4/5
1000/1000 [==============================] - 0s 61us/sample - loss: 0.2055 - acc: 0.9520
Epoch 5/5
1000/1000 [==============================] - 0s 60us/sample - loss: 0.1435 - acc: 0.9710


Now recreate the model from that file:

In [15]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('outputs/my_model.h5')
new_model.summary()

W0429 23:45:36.972855 4562052544 deprecation.py:506] From /Users/shawlu/anaconda3/envs/deep/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0429 23:45:36.974144 4562052544 deprecation.py:506] From /Users/shawlu/anaconda3/envs/deep/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Check its accuracy:

In [16]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 76us/sample - loss: 0.4357 - acc: 0.8690
Restored model, accuracy: 86.90%


This technique saves everything:

* The weight values
* The model's configuration(architecture)
* The optimizer configuration

Keras saves models by inspecting the architecture. Currently, it is not able to save TensorFlow optimizers (from `tf.train`). When using those you will need to re-compile the model after loading, and you will lose the state of the optimizer.


### What's Next

That was a quick guide to saving and loading in with `tf.keras`.

* The [tf.keras guide](https://www.tensorflow.org/guide/keras) shows more about saving and loading models with `tf.keras`.

* See [Saving in eager](https://www.tensorflow.org/guide/eager#object_based_saving) for saving during eager execution.

* The [Save and Restore](https://www.tensorflow.org/guide/saved_model) guide has low-level details about TensorFlow saving.